In [1]:
from sqlalchemy import create_engine
from sqlalchemy.sql.ddl import CreateSchema
from tablas_mfm import *
import numpy as np
import pandas as pd
import generador_filas as gf
from tablas_generacion import *
import datetime


In [2]:
#Devuelve una tabla con nombre "nombre"
def string_to_table(nombre):
    for tb in metadata.sorted_tables:
        if tb.name == nombre:
            return tb
    raise Exception("la tabla "+nombre+" no existe")


In [3]:
#Copia la tabla "nombreTabla" de la BBDD "dbO" a la BBDD "dbD"
def copiarTabla(dbO,dbD, nombreTabla):
    if not isinstance(nombreTabla,Table) :
        raise Exception("la tabla "+nombreTabla+" no existe")
    else :
        result = dbO.execute(nombreTabla.select())
        column=nombreTabla.columns.keys()
        filas=[]
        filasmax=10000
        num_filas=0
        for r in result:
            fila = {}
            for i in range(len(column)):
                fila.update({column[i] : r[i]})
            filas.append(fila)
            num_filas+=1
            if num_filas > filasmax:
                insercion = nombreTabla.insert().values(filas)
                dbD.execute(insercion)
                filas = []
                num_filas=0
        result.close()
        insercion = nombreTabla.insert().values(filas)
        dbD.execute(insercion)


In [11]:
#Crea las conexiones con las BBDD origen y destino
db_destino_string = ""
db_origen_string = ""

dbDestino = create_engine(db_destino_string)
dbOrigen = create_engine(db_origen_string)

dbDes=dbDestino.connect()
dbOr=dbOrigen.connect()

In [6]:
#Crea el esquema para las tablas en la BBDD destino

if not dbDes.dialect.has_schema(dbDes, 'mfm'):
    dbDes.execute(CreateSchema('mfm'))

In [7]:
#Crea las tablas en la BBDD destino
metadata.create_all(dbDes)


In [8]:
#Copia las tablas que sean diccionarios a la base de datos destino
dfs = pd.read_excel('Tablas.xlsx',engine='openpyxl',header=None)
tablas_mfm=dfs.values
for tabla in tablas_mfm:
    if tabla[1] == 'D' :
        copiarTabla(dbOr,dbDes,string_to_table(tabla[0]))



In [5]:
#Cuenta el numero de sectores
tipo_zbs=["RU","UR","CA"]

In [ ]:
res = dbOr.execute(text("select count(*) from mfm.mf_sector"))
nSectores = res.first()[0]
res.close()

sectores = np.zeros([len(tipo_zbs),nSectores])

In [10]:
#Cuenta el numero de zbs segun su tipo que tiene cada sector
q1="select * from (select sector_id , count(*) from mfm.mf_zbs where tipo = '"
q2="' group by sector_id) as zonas order by sector_id;"

for i in range(0,len(tipo_zbs)):
    res= dbOr.execute(text(q1+tipo_zbs[i]+q2))
    for r in res:
        sectores[i,r[0]-1]=r[1]
    res.close()

print(sectores)

[[10. 12.  3. 14.  8. 14. 11.  5.]
 [ 1.  3.  0.  2.  2.  0.  1.  0.]
 [ 3.  0. 18.  6.  0.  2.  0.  8.]]


In [6]:
#Genera los sectores con sus zbs
NUMERO_DE_SECTORES_A_GENERAR=5

In [ ]:
sectores_generados = np.random.multivariate_normal(np.mean(sectores,axis=1), np.cov(sectores), NUMERO_DE_SECTORES_A_GENERAR)
sectores_generados[sectores_generados<0]=0
sectores_generados=np.round(sectores_generados)
print(sectores_generados)

In [12]:
#Cuenta el número zbs totales segun su tipo
q1="select count(*) from mfm.mf_zbs where tipo ='"
q2="';"
nTipo_zbs=[]
for i in range(0,len(tipo_zbs)):
    res= dbOr.execute(text(q1+tipo_zbs[i]+q2))
    nTipo_zbs.append(res.first()[0])
    res.close()
print(nTipo_zbs)

[77, 9, 37]


In [14]:
tipo_AP = ["MEDICINA FAMILIA", "PEDIATRIA AP"]

In [15]:
#Cuenta el numero de cias de AP segun su tipo en cada tipo de zbs
q1="select mz.code, coalesce(_count,0)  from (select zbs_cd as z , count(*) as _count from mfm.mf_cias_v2 where speciality_st = '"
q2="' group by zbs_cd) as s right join mfm.mf_zbs as mz on s.z=mz.code where mz.tipo='"
q3="' order by mz.code;"
cias=[]
for i in range(0,len(tipo_zbs)):
    cias_por_tipo = np.zeros([len(tipo_AP),nTipo_zbs[i]])
    for j in range(0,len(tipo_AP)):
        res= dbOr.execute(text(q1+tipo_AP[j]+q2+tipo_zbs[i]+q3))
        k=0
        for r in res:
            cias_por_tipo[j,k]=r[1]
            k+=1
        res.close()
    cias.append(cias_por_tipo)

In [16]:
#Genero los cias de AP en las zbs creadas
cias_generados=[]
for i in range(0,len(tipo_zbs)):
    cias_generado=np.random.multivariate_normal(np.mean(cias[i],axis=1), np.cov(cias[i]),np.sum(sectores_generados,axis=0).astype(int)[i] )
    cias_generado[cias_generado<0]=0
    cias_generado=np.round(cias_generado)
    cias_generados.append(cias_generado)
#print(cias_generados)


In [18]:
#Inserto en la BBDD destino los sectores,zbs y cias generados
i=0
idsZ=0
idsC=0
indice_tipos_cias=np.zeros([len(tipo_zbs)]).astype(int)
for sector in sectores_generados:
    s = gf.generar_sector(i)
    ins_sec = t_mf_sector.insert().values(s)
    dbDes.execute(ins_sec)
    for k in range(0,len(tipo_zbs)):
        j=0
        while j < sector[k]:
            zbs = gf.generar_zbs(idsZ,tipo_zbs[k],s)
            dbDes.execute(t_mf_zbs.insert().values(zbs))
            cias_g=cias_generados[k][indice_tipos_cias[k]]
            indice_tipos_cias[k]+=1
            l=0
            while l < len(tipo_AP):
                sp={"code": l, "st": tipo_AP[l]}
                m=0
                while m < cias_g[l]:
                    ci = gf.generar_cias(idsC,sp,s,zbs)
                    dbDes.execute(t_mf_cias_v2.insert().values(ci))
                    idsC+=1
                    m+=1
                l+=1
            idsZ+=1
            j+=1
    i+=1

(4265922, 113, 7)
(4681490, 151, 3)
(7192180, 38, 1)
(5810919, 125, 8)
(2904139, 89, 4)
(3148669, 47, 2)
(6421558, 61, 3)
(6882006, 54, 2)
(6304902, 110, 6)
(3647148, 99, 5)
(3357216, 88, 4)
(6603700, 70, 3)
(6178895, 127, 8)
(3097345, 64, 3)
(1552768, 64, 3)
(3273251, 87, 4)
(7123544, 59, 3)
(3852412, 84, 4)
(6442892, 43, 1)
(6140270, 132, 8)
(243606, 139, 4)
(6383038, 88, 4)
(6525350, 100, 6)
(7023490, 72, 4)
(6471790, 45, 2)
(6088183, 36, 1)
(94404, 41, 1)
(7266853, 125, 8)
(4757122, 130, 8)
(6977024, 65, 3)
(1754388, 107, 6)
(3947891, 105, 6)
(4436455, 88, 4)
(2441094, 67, 3)
(7248848, 39, 1)
(1640240, 60, 3)
(1619823, 109, 6)
(6075973, 43, 1)
(6497169, 47, 2)
(3681281, 79, 4)
(3157974, 55, 2)
(1650671, 130, 8)
(6044420, 58, 3)
(4112448, 62, 3)
(2346787, 60, 3)
(6867312, 117, 7)
(5923575, 94, 5)
(6202120, 68, 3)
(833221, 67, 3)
(6016036, 39, 1)
(1804717, 33, 3)
(5873387, 67, 3)
(6503260, 40, 1)
(2596109, 83, 4)
(679988, 62, 3)
(6588108, 122, 8)
(6113428, 152, 3)
(6135916, 38, 1)
(2

In [23]:
#Crea la tabla con la equivalencia entre los antiguos y nuevos usuarios
if not dbDes.dialect.has_schema(dbDes, 'gn'):
    dbDes.execute(CreateSchema('gn'))
metadata_g.create_all(dbDes)

In [18]:
query="select * from (select id, sexo, nacimiento_dt, zbs, sector_cd from mfm.mf_usuarios where active=true) as us join"\
      "(select id, tipo from mfm.mf_zbs) as zb on zb.id=us.zbs;"
res = dbOr.execute(text(query))
i=0
for r in res:
    print(r)

    #dias desplazados
    dias=np.random.randint(0,365)
    nacimiento=r[2]-datetime.timedelta(days=dias)

    #selecciona el tipo de zbs
    tipo=tipo_zbs.index(r[6])
    u = np.random.randint(1,10)
    aux=0
    if u<8:
        aux=0
    elif u<9:
        aux=1
    else:
        aux=2

    tipo_n=(tipo+aux) % 3

    #selecciona nuevo sector
    noseleccionado=True
    sector=np.random.randint(0,NUMERO_DE_SECTORES_A_GENERAR-1)
    while noseleccionado:
        query="select count(*) from mfm.mf_zbs where sector_id="+str(sector)+" and tipo='"+tipo_zbs[tipo_n]+"';"
        if dbDes.execute(text(query)).first()[0]==0:
            sector=(sector+1) % NUMERO_DE_SECTORES_A_GENERAR
        else:
            noseleccionado=False

    #selecciona zbs
    query="select min(cast(id as int)),max(cast(id as int)) from mfm.mf_zbs where sector_id="+str(sector)+" and tipo='"+tipo_zbs[tipo_n]+"';"
    res_zbs=dbDes.execute(text(query))
    res_zbs_id=res_zbs.first()
    minimo=int(res_zbs_id[0])
    maximo=int(res_zbs_id[1])
    print(minimo)
    print(maximo)
    if minimo==maximo:
        zbs=res_zbs_id[0]
    else:
        zbs=np.random.randint(res_zbs_id[0],res_zbs_id[1])

    #selecciona cias
    query="select min(cast(code as int)), max(cast(code as int)) from mfm.mf_cias_v2 where zbs_cd='"+str(zbs)+"';"
    res_cias=dbDes.execute(text(query))
    res_cias_id=res_cias.first()
    minimo=int(res_cias_id[0])
    maximo=int(res_cias_id[1])
    print(zbs)
    print(minimo)
    print(maximo)
    if minimo==maximo:
        cias=res_cias_id[0]
    else:
        cias=np.random.randint(res_cias_id[0],res_cias_id[1])


    #inserta el nuevo usuario en la BD destino e inserta la equivalencia con el antiguo usuario
    us = gf.generar_usuario(i,r[1],nacimiento,zbs,sector,cias)
    ins_us = t_mf_usuarios.insert().values(us)
    eq = gf.generar_eq(r[0],i,dias)
    ins_eq = t_gn_equivalente.insert().values(eq)
    dbDes.execute(ins_us)
    dbDes.execute(ins_eq)
    i+=1
res.close()


(4265922, 2, datetime.date(1951, 9, 9), 113, 7, 113, 'RU')
20
28
27
216
225
(4681490, 2, datetime.date(1953, 1, 22), 151, 3, 151, 'CA')
31
32
31
265
283
(7192180, 2, datetime.date(1999, 1, 9), 38, 1, 38, 'CA')
31
32
31
265
283
(5810919, 2, datetime.date(1970, 5, 28), 125, 8, 125, 'CA')
53
63
61
548
554
(2904139, 2, datetime.date(1980, 11, 3), 89, 4, 89, 'CA')
31
32
31
265
283
(3148669, 1, datetime.date(1966, 8, 10), 47, 2, 47, 'UR')
64
66
64
574
590
(6421558, 1, datetime.date(1952, 5, 18), 61, 3, 61, 'CA')
31
32
31
265
283
(6882006, 2, datetime.date(1992, 1, 19), 54, 2, 54, 'UR')
16
19
18
146
158
(6304902, 1, datetime.date(1914, 6, 16), 110, 6, 110, 'CA')
50
52
51
458
471
(3647148, 1, datetime.date(1951, 2, 10), 99, 5, 99, 'RU')
33
47
42
368
375
(3357216, 1, datetime.date(1960, 6, 13), 88, 4, 88, 'CA')
31
32
31
265
283
(6603700, 1, datetime.date(1949, 6, 28), 70, 3, 70, 'CA')
31
32
31
265
283
(6178895, 1, datetime.date(2005, 9, 23), 127, 8, 127, 'CA')
31
32
31
265
283
(3097345, 2, date

In [12]:
dbDes.close()
dbDestino.dispose()
dbOr.close()
dbOrigen.dispose()